In [49]:
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map 
import numpy as np
import random
from prettytable import PrettyTable

dice = [1, 2, 3, 4, 5, 6]   #my dice

ID_dict={"x_00":1,"x_20":2,"x_22":3,"x_32":4,"x_30":5,"x_40":6,"x_50":7} #ID all the states with 1-7
def common(x):
    "Function that determines which face value appears most"
    l=[x.count(1), x.count(2), x.count(3), x.count(4), x.count(5), x.count(6)]
    A = np.argmax(l)+1
    return A

def State(l):
    "Function takes list l"
    "return state of a roll"
    if max(l)==5:                #check if its AAAAA
        state="x_50"
    elif max(l)==4:              #check if its AAAAB
        state="x_40"
    elif max(l)==3:              #check if its AAABB or AAABC
        new_l= l
        # removing the largest element from temp list
        new_l.remove(max(new_l))
        if max(new_l)==2:        #check if its AAABB  
            state="x_32"
        else:                    #else its AAABC   
            state="x_30"
    elif max(l)==2:
        new_l= l
        # removing the largest element from temp list
        new_l.remove(max(new_l)) 
        if max(new_l)==2:        #check if its AABBC
            state="x_22"
        else:                    #else its AABCD
            state="x_20"
    
    
    else:                        #else its ABCDE
        state="x_00"
    return state    
  
def Re_Roll(state,A):
        "Function takes state of a list (state) and the most appearing number (A) " 
        "returns a list containing new state and and the same A" 
        if state=="x_50":
          x=[A,A,A,A,A]
          l=[x.count(1), x.count(2), x.count(3), x.count(4), x.count(5), x.count(6)]
          state=State(l)
        elif state=="x_40": 
          x=[A,A,A,A]
          x_ = random.choices(dice,k=1)
          x.append(x_[0])
          l=[x.count(1), x.count(2), x.count(3), x.count(4), x.count(5), x.count(6)]
          state=State(l)   
        elif state=="x_30":
          x=[A,A,A]
          x_ = random.choices(dice,k=2)
          x.append(x_[0])
          x.append(x_[1])
          l=[x.count(1), x.count(2), x.count(3), x.count(4), x.count(5), x.count(6)]
          state=State(l)
        elif state=="x_32":
          x=[A,A,A]
          x_ = random.choices(dice,k=2)
          x.append(x_[0])
          x.append(x_[1])
          l=[x.count(1), x.count(2), x.count(3), x.count(4), x.count(5), x.count(6)]
          state=State(l)
        elif state=="x_22":
          x=[A,A]
          x_ = random.choices(dice,k=3)
          x.append(x_[0])
          x.append(x_[1])
          x.append(x_[2])
          l=[x.count(1), x.count(2), x.count(3), x.count(4), x.count(5), x.count(6)]
          state=State(l)
        elif state=="x_20":
          x=[A,A]
          x_ = random.choices(dice,k=3)
          x.append(x_[0])
          x.append(x_[1])
          x.append(x_[2])
          l=[x.count(1), x.count(2), x.count(3), x.count(4), x.count(5), x.count(6)]
          state=State(l)
        else:
          x= random.choices(dice,k=5)
          l=[x.count(1), x.count(2), x.count(3), x.count(4), x.count(5), x.count(6)]
          state=State(l)
        return [state, A] 
for roll in range(3):          
  def results(j):
    
      "Returns the state ID of of the list i.e. x_00, x_20, x_22, x_32, x_30, x_40, x_50"
      x = random.choices(dice,k=5)
      #count the number of 1s 2s 3s 4s 5s 6s and store in a list l
      l=[x.count(1), x.count(2), x.count(3), x.count(4), x.count(5), x.count(6)]
      state=State(l)
      A=common(x) #determine which number appears most   
      for r in range(roll):
        R=Re_Roll(state, A)
        state=R[0]
        A=R[1]
      return ID_dict[state]      #return the state ID


  reps = int(1e6) # number of repetions of game
  cores = 2 # number of cores to be used in parallel 

  # run the function results reps times (chunksize is how many executions are clustered together for one run on a single core)
  res = process_map( results, range(reps), max_workers=cores, total=reps, chunksize=int(reps/1000), tqdm_class=tqdm)


  # print how often we got which state
  print("Roll number ",roll+1," probabilities.")
  print('[x_00, x_20, x_22, x_32, x_30, x_40, x_50]=', np.bincount(res)[1:] / reps, "respectively." )
  print("____________________________________________________________________________________________")


100%|██████████| 1000000/1000000 [00:14<00:00, 71267.84it/s]


Roll number  1  probabilities.
[x_00, x_20, x_22, x_32, x_30, x_40, x_50]= [0.092947 0.462249 0.231824 0.038748 0.154059 0.019445 0.000728] respectively.
____________________________________________________________________________________________


100%|██████████| 1000000/1000000 [00:21<00:00, 46600.24it/s]

Roll number  2  probabilities.
[x_00, x_20, x_22, x_32, x_30, x_40, x_50]= [0.008558 0.234816 0.214649 0.094911 0.314396 0.120199 0.012471] respectively.
____________________________________________________________________________________________



100%|██████████| 1000000/1000000 [00:28<00:00, 34908.71it/s]


Roll number  3  probabilities.
[x_00, x_20, x_22, x_32, x_30, x_40, x_50]= [0.000812 0.129353 0.126692 0.098601 0.353642 0.244881 0.046019] respectively.
____________________________________________________________________________________________
